The goal of this notebook is to query the rich dataset of nycflights13, and to recreate tables and calculations from my stats306 repository detailing problems from this undergraduate class.

In [2]:
import pandas as pd
import numpy as np
import nycflights13 as nyc
from nycflights13 import planes
from nycflights13 import airports
import pyreadr
import duckdb

In [3]:
flights = nyc.flights
airlines = nyc.airlines

In [4]:
flights.size

6398744

In [5]:
flights.columns

Index(['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay',
       'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight',
       'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute',
       'time_hour'],
      dtype='object')

In [6]:
airlines.size

32

In [7]:
airlines.columns

Index(['carrier', 'name'], dtype='object')

## Problem 1:
Extract all the flights that took place in months that begin with the letter "A"

In [8]:
# flights that took place in months starting with the letter "A" occurred in either April (month 4) or August (month 8)
q1results = duckdb.query("select * from flights where month in (4, 8)") 
q1results

┌───────┬───────┬───────┬──────────┬────────────────┬───┬──────────┬──────────┬───────┬────────┬──────────────────────┐
│ year  │ month │  day  │ dep_time │ sched_dep_time │ … │ air_time │ distance │ hour  │ minute │      time_hour       │
│ int64 │ int64 │ int64 │  double  │     int64      │   │  double  │  int64   │ int64 │ int64  │       varchar        │
├───────┼───────┼───────┼──────────┼────────────────┼───┼──────────┼──────────┼───────┼────────┼──────────────────────┤
│  2013 │     4 │     1 │    454.0 │            500 │ … │     84.0 │      529 │     5 │      0 │ 2013-04-01T09:00:00Z │
│  2013 │     4 │     1 │    509.0 │            515 │ … │    194.0 │     1400 │     5 │     15 │ 2013-04-01T09:00:00Z │
│  2013 │     4 │     1 │    526.0 │            530 │ … │    206.0 │     1416 │     5 │     30 │ 2013-04-01T09:00:00Z │
│  2013 │     4 │     1 │    534.0 │            540 │ … │    152.0 │     1089 │     5 │     40 │ 2013-04-01T09:00:00Z │
│  2013 │     4 │     1 │    542.0 │    

# Problem 2

Find the number of United Airlines flights that departed exactly on time and arrived exactly on time.

In [9]:
q2result = duckdb.query("select count(*) as problemTwoAnswer from flights where (carrier = 'UA' and (dep_delay == 0 and arr_delay == 0))")
q2result

┌──────────────────┐
│ problemTwoAnswer │
│      int64       │
├──────────────────┤
│               58 │
└──────────────────┘

# Problem 3

Find all flights destined for LAX or SFO which departed more than an hour late, but did not arrive late.

In [10]:
q3results = duckdb.query("select * from flights where ((dest = 'LAX' OR dest = 'SFO') AND dep_delay >= 60 AND (arr_delay < 0))")
q3results

┌───────┬───────┬───────┬──────────┬────────────────┬───┬──────────┬──────────┬───────┬────────┬──────────────────────┐
│ year  │ month │  day  │ dep_time │ sched_dep_time │ … │ air_time │ distance │ hour  │ minute │      time_hour       │
│ int64 │ int64 │ int64 │  double  │     int64      │   │  double  │  int64   │ int64 │ int64  │       varchar        │
├───────┼───────┼───────┼──────────┼────────────────┼───┼──────────┼──────────┼───────┼────────┼──────────────────────┤
│  2013 │     4 │     5 │    932.0 │            831 │ … │    301.0 │     2454 │     8 │     31 │ 2013-04-05T12:00:00Z │
├───────┴───────┴───────┴──────────┴────────────────┴───┴──────────┴──────────┴───────┴────────┴──────────────────────┤
│ 1 rows                                                                                        19 columns (10 shown) │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

# Problem 4

Find the number of flights in May that are missing their departure time or their arrival time, but not both. 

In [11]:
q4results = duckdb.query("SELECT count(*) as probFourCount from flights \
                         WHERE month = 5 AND ((arr_time IS NOT NULL) <> (dep_time is NOT NULL))") # <> denotes xor
q4results

┌───────────────┐
│ probFourCount │
│     int64     │
├───────────────┤
│            38 │
└───────────────┘

# Problem 5

There are four flights which have a tail number that does not begin with the letter "N". Find those 

In [12]:
q5results = duckdb.query("SELECT * FROM flights where tailnum not like 'N%'")
q5results

┌───────┬───────┬───────┬──────────┬────────────────┬───┬──────────┬──────────┬───────┬────────┬──────────────────────┐
│ year  │ month │  day  │ dep_time │ sched_dep_time │ … │ air_time │ distance │ hour  │ minute │      time_hour       │
│ int64 │ int64 │ int64 │  double  │     int64      │   │  double  │  int64   │ int64 │ int64  │       varchar        │
├───────┼───────┼───────┼──────────┼────────────────┼───┼──────────┼──────────┼───────┼────────┼──────────────────────┤
│  2013 │     2 │    11 │   1508.0 │           1400 │ … │    131.0 │      762 │    14 │      0 │ 2013-02-11T19:00:00Z │
│  2013 │     3 │    23 │   1340.0 │           1300 │ … │    153.0 │      950 │    13 │      0 │ 2013-03-23T17:00:00Z │
│  2013 │     3 │    24 │    859.0 │            835 │ … │    143.0 │      944 │     8 │     35 │ 2013-03-24T12:00:00Z │
│  2013 │     7 │     5 │   1253.0 │           1259 │ … │    112.0 │      762 │    12 │     59 │ 2013-07-05T16:00:00Z │
├───────┴───────┴───────┴──────────┴────

# Problem 6

Find the unique tail numbers(s) of the flights that were in the air for the second shortest and third longest amounts of time. Store them in variables shortest2_tail_num and longest3_tail_num, respectively.

--> Running into the issue: there is no guarantee that an R function can be translated into sql code
- To solve this issue, instead of using R's mutate and min_rank, I found the second shortest and third longest air_time values and plugged those into a where clause

In [13]:
q6results_secondShortest = duckdb.query("SELECT distinct(tailnum) from \
                                        flights WHERE air_time = (SELECT min(air_time) as secondShortest \
                                        from flights where air_time > (select min(air_time) from flights))")


q6results_secondShortest

┌─────────┐
│ tailnum │
│ varchar │
├─────────┤
│ N947UW  │
│ N29906  │
│ N12195  │
│ N16987  │
│ N11191  │
│ N12160  │
│ N11109  │
│ N27200  │
│ N13913  │
│ N18101  │
│ N12921  │
│ N14920  │
│ N13955  │
│ N8501F  │
├─────────┤
│ 14 rows │
└─────────┘

In [14]:
q6results_thirdLongest = duckdb.query("SELECT distinct(tailnum) from flights \
                                      WHERE air_time = (SELECT air_time from flights ORDER BY air_time DESC LIMIT 1 OFFSET 2)")
q6results_thirdLongest

┌─────────┐
│ tailnum │
│ varchar │
├─────────┤
│ N380HA  │
│ N388HA  │
└─────────┘

# Problem 7

Sort the rows of flights such that the months are arranged in the following order: spring, summer, fall, winter. (Here we define winter to be January-March, spring is April-June, etc.) Within each season, the months should be sorted in ascending order. After sorting, drop all columns except for month, day and tail number. Store the sorted and subsetted table in a variable called table7.

In [15]:
q7result = duckdb.query("SELECT month, day, tailnum from flights ORDER BY case month \
                            when '4' then 1 \
                            when '5' then 2 \
                            when '6' then 3 \
                            when '7' then 4 \
                            when '8' then 5 \
                            when '9' then 6 \
                            when '10' then 7 \
                            when '11' then 8 \
                            when '12' then 9 \
                            when '1' then 10 \
                            when '2' then 11 \
                            when '3' then 12 \
                        end, \
                        month")
q7result

┌───────┬───────┬─────────┐
│ month │  day  │ tailnum │
│ int64 │ int64 │ varchar │
├───────┼───────┼─────────┤
│     4 │     1 │ N566UW  │
│     4 │     1 │ N76288  │
│     4 │     1 │ N76517  │
│     4 │     1 │ N5DSAA  │
│     4 │     1 │ N784JB  │
│     4 │     1 │ N590JB  │
│     4 │     1 │ N737US  │
│     4 │     1 │ N750UW  │
│     4 │     1 │ N510MQ  │
│     4 │     1 │ N723EV  │
│     · │     · │   ·     │
│     · │     · │   ·     │
│     · │     · │   ·     │
│     4 │    11 │ N826AS  │
│     4 │    11 │ N11137  │
│     4 │    11 │ N604DL  │
│     4 │    11 │ N17126  │
│     4 │    11 │ N5CHAA  │
│     4 │    11 │ N584AA  │
│     4 │    11 │ N249JB  │
│     4 │    11 │ N284WN  │
│     4 │    11 │ N823UA  │
│     4 │    11 │ N17105  │
├───────┴───────┴─────────┤
│ ? rows        3 columns │
└─────────────────────────┘

# Problem 8

Drop the even-numbered columns of flights. Store the resulting data table in a variable called table8.

In [16]:
q8result = duckdb.query("SELECT year, day, sched_dep_time, arr_time, arr_delay, flight, origin, air_time, hour, time_hour from flights") 
# no function to drop even numbered cols in SQL, so I hard-coded this one
q8result

┌───────┬───────┬────────────────┬──────────┬───────────┬────────┬─────────┬──────────┬───────┬──────────────────────┐
│ year  │  day  │ sched_dep_time │ arr_time │ arr_delay │ flight │ origin  │ air_time │ hour  │      time_hour       │
│ int64 │ int64 │     int64      │  double  │  double   │ int64  │ varchar │  double  │ int64 │       varchar        │
├───────┼───────┼────────────────┼──────────┼───────────┼────────┼─────────┼──────────┼───────┼──────────────────────┤
│  2013 │     1 │            515 │    830.0 │      11.0 │   1545 │ EWR     │    227.0 │     5 │ 2013-01-01T10:00:00Z │
│  2013 │     1 │            529 │    850.0 │      20.0 │   1714 │ LGA     │    227.0 │     5 │ 2013-01-01T10:00:00Z │
│  2013 │     1 │            540 │    923.0 │      33.0 │   1141 │ JFK     │    160.0 │     5 │ 2013-01-01T10:00:00Z │
│  2013 │     1 │            545 │   1004.0 │     -18.0 │    725 │ JFK     │    183.0 │     5 │ 2013-01-01T10:00:00Z │
│  2013 │     1 │            600 │    812.0 │   

# Problem 9

Define a flight to be "spooky" if it was in the air at 1:13pm on either Halloween or Friday, September 13th. There were 110 spooky flights on Friday the 13th. How many spooky flights are there in total? Store your answer in a variable called nspooky.

In [17]:
q9result = duckdb.query("SELECT count(*) as nspooky from flights \
                        where ((day = 13 OR day = 31) AND (month = 9 or month = 10) \
                        AND (1313 < arr_time AND 1313 > dep_time))")
q9result

┌─────────┐
│ nspooky │
│  int64  │
├─────────┤
│     318 │
└─────────┘

# Problem 10

Select all the columns in flights which end with "delay". In the resulting table, define a new column max_delay which equals the maximum of the departure and arrival delays for each flight. For example, if a flight had a departure delay of -1 and an arrival delay of 10, the max_delay would equal 10. Sort your table in descending order of max_delay and store it in a variable called table10.

In [18]:
q10result = duckdb.query("SELECT dep_delay, arr_delay, GREATEST(dep_delay, arr_delay) as max_delay \
                         from flights ORDER BY max_delay DESC ") # unfortunately the LIKE operator works within columns, not on column names
q10result

┌───────────┬───────────┬───────────┐
│ dep_delay │ arr_delay │ max_delay │
│  double   │  double   │  double   │
├───────────┼───────────┼───────────┤
│    1301.0 │    1272.0 │    1301.0 │
│    1137.0 │    1127.0 │    1137.0 │
│    1126.0 │    1109.0 │    1126.0 │
│    1014.0 │    1007.0 │    1014.0 │
│    1005.0 │     989.0 │    1005.0 │
│     960.0 │     931.0 │     960.0 │
│     911.0 │     915.0 │     915.0 │
│     899.0 │     850.0 │     899.0 │
│     898.0 │     895.0 │     898.0 │
│     896.0 │     878.0 │     896.0 │
│       ·   │       ·   │       ·   │
│       ·   │       ·   │       ·   │
│       ·   │       ·   │       ·   │
│     112.0 │     129.0 │     129.0 │
│     129.0 │     108.0 │     129.0 │
│     129.0 │     124.0 │     129.0 │
│     114.0 │     129.0 │     129.0 │
│     129.0 │     123.0 │     129.0 │
│     103.0 │     129.0 │     129.0 │
│     129.0 │      89.0 │     129.0 │
│     126.0 │     129.0 │     129.0 │
│     129.0 │     111.0 │     129.0 │
│     101.0 

This is the end of the assignments found in https://github.com/kaspersj/Stats-306/blob/master/problemset2.ipynb. 

### Looking now at Problem Set 3:
Found at https://github.com/kaspersj/Stats-306/blob/master/problemset3.ipynb, we can query and find solutions to problems: 1, 2, 4 and 5.


In [19]:
bil = pyreadr.read_r('bil.RData')

bil = bil['bil']
print(type(bil))

<class 'pandas.core.frame.DataFrame'>


In [20]:
bil['age'].value_counts()

age
-1     385
 60     76
 64     65
 50     65
 69     65
      ... 
 95      2
 96      2
 12      1
 21      1
 98      1
Name: count, Length: 74, dtype: int64

Looking at age, we see many values at -1. These represent missing data where we do not know the person's age. We'll fix this by recoding all values of -1 to NA.

In [21]:
bil['age'] = bil['age'].replace(-1, "NA")
bil

,age,category,citizenship,company.name,company.type,country code,founded,from emerging,gdp,gender,...,name,rank,region,relationship,sector,was founder,was political,wealth.type,worth in billions,year
0,NA,Financial,Saudi Arabia,Rolaco Trading and Contracting Company,new,SAU,1968,True,1.580000e+11,male,...,Abdul Aziz Al-Sulaiman,404,Middle East/North Africa,founder,construction,True,False,self-made finance,1.0,1996
1,34,Financial,United States,Fidelity Investments,new,USA,1946,True,8.100000e+12,female,...,Abigail Johnson,145,North America,relation,investment banking,True,False,inherited,2.5,1996
2,59,Non-Traded Sectors,Brazil,Companhia Brasileira de Distribui?ao,new,BRA,1948,True,8.540000e+11,male,...,Abilio dos Santos Diniz,322,Latin America,relation,retail,True,False,inherited,1.2,1996
3,61,New Sectors,Germany,Ratiopharm,new,DEU,1881,True,2.500000e+12,male,...,Adolf Merckle,388,Europe,relation,pharmaceuticals,True,False,inherited,1.0,1996
4,NA,Financial,Hong Kong,Swire,new,HKG,1816,True,1.600000e+11,male,...,Adrian and John Swire,162,East Asia,relation,trading company,True,False,inherited,2.2,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,49,Financial,China,Yurun Food,new,CHN,1993,True,0.000000e+00,male,...,Zhu Yicai,1154,East Asia,founder,meat products,True,False,self-made finance,1.5,2014
2610,48,Non-Traded Sectors,Russia,Stroygazconsulting,new,RUS,1992,True,0.000000e+00,male,...,Ziyad Manasir,609,Europe,founder,construction,True,True,privatized and resources,2.8,2014
2611,45,Resource Related,Russia,Summa Group,new,RUS,2004,True,0.000000e+00,male,...,Ziyaudin Magomedov,1372,Europe,founder,"ports, gas",True,False,privatized and resources,1.2,2014
2612,68,Traded Sectors,China,Hangzhou Wahaha Group,new,CHN,1987,True,0.000000e+00,male,...,Zong Qinghou,94,East Asia,founder,beverages,True,False,founder non-finance,11.6,2014


# Problem 1
In which country are billionaires oldest on average? Youngest? Do not count any country which has less than five observations. Store your answers in variables oldest1 and youngest1, respectively.

In [22]:
pset3prob1_youngest = duckdb.query(" \
                          WITH country_averages AS ( \
  SELECT citizenship, ROUND(AVG(CAST(age AS DOUBLE)), 2) AS average_age \
  FROM bil \
  WHERE age != 'NA' \
  GROUP BY citizenship \
  HAVING COUNT(*) > 5 \
) \
SELECT citizenship AS youngest1 \
FROM country_averages \
ORDER BY average_age ASC \
LIMIT 1")

pset3prob1_youngest # should be "United Arab Emirates"

┌───────────┐
│ youngest1 │
│  varchar  │
├───────────┤
│ Venezuela │
└───────────┘

In [23]:
pset3prob1_oldest = duckdb.query(" \
                          WITH country_averages AS ( \
  SELECT citizenship, ROUND(AVG(CAST(age AS DOUBLE)), 2) AS average_age \
  FROM bil \
  WHERE age != 'NA' \
  GROUP BY citizenship \
  HAVING COUNT(*) >= 5 \
) \
SELECT citizenship AS oldest1 \
FROM country_averages \
ORDER BY average_age DESC \
LIMIT 1")
pset3prob1_oldest

┌──────────┐
│ oldest1  │
│ varchar  │
├──────────┤
│ Portugal │
└──────────┘

# Problem 2:
Each billionaire has a category indicating how they made their fortune. Group the billionaires into three age brackets: "40 and under", "41 to 65", and "above 65". (There are just 77 billionaires aged 40 and under, for example.) What is the most common category of billionaire in each of the three age brackets? Store your answer in table2. The table should have three rows (one per category) and three columns: age_bracket, most_common_category and n, the number of billionaires in the most common category.

## Problem 3: 
Not a SQL problem...

## Problem 4:

Recall that each airplane has a unique tail number given by tailnum. Find the tail number of the airplane which flew to the largest number of unique destinations from any of the the three departure airports in flights. Store the string containing this tail number in a variable called most_dests.

In [24]:
pset3prob4 = duckdb.query("SELECT tailnum, COUNT(DISTINCT(dest)) as n from flights \
                                                      WHERE tailnum IS NOT NULL \
                          GROUP BY tailnum \
                          ORDER BY n DESC \
                          LIMIT 1")
pset3prob4

┌─────────┬───────┐
│ tailnum │   n   │
│ varchar │ int64 │
├─────────┼───────┤
│ N11194  │    47 │
└─────────┴───────┘

### Problem 5:
Let a flight's "positive arrival delay" be defined as the larger of arr_delay and zero. We say a flight is ridiculously late if its arrival delay was more than ten times the average positive arrival delay for all flights in that week.

Use the flights_week table to calculate the number of ridiculously late flights in each week of the year. For example, in the first week of the year there were 81 ridiculously late flights.
Also add in the total number of flights in the data set for each week.
Sort the resulting table in descending order of the number of ridiculously late flights and store it in a variable called table5. The table should have three columns, week, n, and n_ridiculously_late.

In [25]:
# flights['time_hour']

In [26]:
# duckdb.query("SELECT DATEPART(WEEK, CONVERT(DATE, time_hour)) - 1 AS week_number from flights")

## Looking now at Problem Set 4:
Found at https://github.com/kaspersj/Stats-306/blob/master/problem%20set%204.ipynb, we can query and find solutions to problems: 2, 3, 4, 5, 6, and 11.


# Problem 2: 
The most common model of airplane flown by United Airlines is the 737-824; there are 122 instances of this type of aircraft in the data. What is the most common airplane model flown by Delta airlines? Store your answer in a variable called model2.

In [27]:
pset4prob2example = duckdb.query("SELECT planes.model FROM flights \
                                 LEFT JOIN planes \
                                 ON planes.tailnum = flights.tailnum \
                                 WHERE carrier = 'UA' \
                                 GROUP BY model \
    ORDER BY COUNT(*) DESC \
    LIMIT    1;")
pset4prob2example

┌─────────┐
│  model  │
│ varchar │
├─────────┤
│ 737-824 │
└─────────┘

In [28]:
pset4prob2 = duckdb.query("SELECT planes.model FROM flights \
                                 LEFT JOIN planes \
                                 ON planes.tailnum = flights.tailnum \
                                 WHERE carrier = 'DL' \
                                 GROUP BY model \
                                ORDER BY COUNT(*) DESC \
                                LIMIT    1;")
pset4prob2

┌─────────┐
│  model  │
│ varchar │
├─────────┤
│ MD-88   │
└─────────┘

# Problem 3

In July there were 31 departures from JFK destined for somewhere tropical. How many were there in December? Store your answer in a variable called lga3.

In [29]:
pset4prob3 = duckdb.query("SELECT count(*) from flights \
                          LEFT JOIN airports \
                          ON airports.faa = flights.dest \
                          WHERE (flights.origin = 'JFK' AND flights.month = 12 AND airports.lat between -23.5 AND 23.5 ) \
                         ;")
pset4prob3

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           28 │
└──────────────┘

# Problem 4
There were 27261 United Airlines flights which experienced a departure delay:

In [30]:
pset4prob4 = duckdb.query("SELECT count(*) as UA_dep_delay from flights \
                          WHERE (carrier = 'UA' AND dep_delay > 0) \
                         ;")
pset4prob4

┌──────────────┐
│ UA_dep_delay │
│    int64     │
├──────────────┤
│        27261 │
└──────────────┘

Of the 27261 flights that experienced a departure delay, and after dropping flights with missing dep_delay, it was also the case that the immediately preceding (in chronological order, with ties broken by tail number) United Airlines flight was also delayed. Hence, the odds of experiencing a delay given that the previous flight was also delayed are: 15246 / (27261 - 15246) = 1.27

Using the same criteria, what are the odds that a Delta Airlines flight is delayed given that the previous Delta flight was also delayed? Store your answer in a random variable called odds4. (We will check your to two decimal places.)

In [31]:
odds4 = duckdb.query("SELECT count(*) from flights \
                     WHERE (dep_delay > 0 AND carrier = 'DL') \
                     ;")

odds4

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        15241 │
└──────────────┘

In [32]:
pset4prob5 = 15241 / (27261 - 15241)
pset4prob5

1.2679700499168054

### Looking now at Problem Set 5:
Found at https://github.com/kaspersj/Stats-306/blob/master/problemset5.ipynb, we can query and find solutions to problems: 1, 2, 3, 4 and 5.


UCBAdmissions data can be found here:
https://github.com/vincentarelbundock/Rdatasets/blob/master/csv/datasets/UCBAdmissions.csv


In [33]:
ucb = pd.read_csv("UCBAdmissions.csv")
ucb.head(3)

,rownames,Admit,Gender,Dept,Freq
0,1,Admitted,Male,A,512
1,2,Rejected,Male,A,313
2,3,Admitted,Female,A,89


# Problem 1: 

Using the tool(s) we learned for summarizing and manipulating tidy data, create a summary table from ucb which shows the acceptance rate by gender. Your table should have four columns: Gender, Accepted, Rejected, and Acceptance Proportion (the fraction of accepted applicants within each gender) and should look like this:

Gender	Admitted	Rejected	Acceptance Proportion
Female	557	&dash;	&dash;
Male	&dash;	&dash;	0.4451877
(A few entries have been provided for you; your job is to write code that will produce the complete table with no blanks.)

Store the resulting tibble in a variable called table1.

In [34]:
pset5_prob1 = duckdb.query("WITH Admitted AS  (\
                           SELECT Gender, sum(Freq) as admitted_sum from ucb \
                           WHERE Admit = 'Admitted' \
                           GROUP BY Gender \
                           ) \
SELECT * FROM Admitted")

pset5_prob1

┌─────────┬──────────────┐
│ Gender  │ admitted_sum │
│ varchar │    int128    │
├─────────┼──────────────┤
│ Female  │          557 │
│ Male    │         1198 │
└─────────┴──────────────┘

In [35]:
pset5_prob1part2 = duckdb.query("WITH Rejected AS  (\
                           SELECT Gender, sum(Freq) as rejected_sum from ucb \
                           WHERE Admit = 'Rejected' \
                           GROUP BY Gender \
                           ) \
SELECT * FROM Rejected")

pset5_prob1part2

┌─────────┬──────────────┐
│ Gender  │ rejected_sum │
│ varchar │    int128    │
├─────────┼──────────────┤
│ Male    │         1493 │
│ Female  │         1278 │
└─────────┴──────────────┘

In [36]:
pset5_prob1part3 = duckdb.query("WITH table1 as ( \
                                SELECT * FROM pset5_prob1part2 \
                                JOIN pset5_prob1 \
                                ON pset5_prob1.Gender = pset5_prob1part2.Gender \
                                ) \
    SELECT Gender, rejected_sum, admitted_sum, (admitted_sum)/(admitted_sum + rejected_sum) AS acceptance_proportion from table1")

pset5_prob1part3

┌─────────┬──────────────┬──────────────┬───────────────────────┐
│ Gender  │ rejected_sum │ admitted_sum │ acceptance_proportion │
│ varchar │    int128    │    int128    │        double         │
├─────────┼──────────────┼──────────────┼───────────────────────┤
│ Male    │         1493 │         1198 │    0.4451876625789669 │
│ Female  │         1278 │          557 │   0.30354223433242505 │
└─────────┴──────────────┴──────────────┴───────────────────────┘

## Problem 2:
In STATS 250 you learned how to test for differences in proportions between two populations. Apply this to Problem 1. Was the overall acceptance proportion for men statistically different from that of women? Perform an appropriate test and interpret your findings. What do these result suggest about admissions practices at UC Berkeley in the early 1970s?

*Interpretation*

To test for significance we must do a two proportion z test with a hypothesis of p1=p2. To perform this we must first find the general p which is = (557 +1198/ 557+1278+1198+1493). This equals .63345. We are provided p1 and p2 by the Acceptance rate proportion. p1 = .4451877 & p2 = .303355422. We then plug these values into the equation z = (p1-p2)/sqrt(p(1-p)(1/n1+1/n2) where p = the general p described above and n is that sample sizes of men and female. We get a z score of 9.7. This corresponds to a p value of <.001 which is less than .05, so the overall acceptance proportion for men was statistically different from that of women in UC Berkeley in the early 1970s.

##  Problem 3:

Reproduce the table from Problem 1, but now stratify by department. Compute the male and female acceptance proportion for each department separately. Your resulting table should look like:

In [37]:
pset5_prob3 = duckdb.query("WITH rejected AS  (\
                           SELECT Dept, Gender, sum(Freq) as rejected_sum from ucb \
                           WHERE Admit = 'Rejected' \
                           GROUP BY Dept, Gender \
                           ) \
SELECT * FROM rejected")

pset5_prob3

┌─────────┬─────────┬──────────────┐
│  Dept   │ Gender  │ rejected_sum │
│ varchar │ varchar │    int128    │
├─────────┼─────────┼──────────────┤
│ F       │ Female  │          317 │
│ E       │ Female  │          299 │
│ B       │ Male    │          207 │
│ C       │ Male    │          205 │
│ D       │ Male    │          279 │
│ A       │ Female  │           19 │
│ E       │ Male    │          138 │
│ B       │ Female  │            8 │
│ C       │ Female  │          391 │
│ D       │ Female  │          244 │
│ F       │ Male    │          351 │
│ A       │ Male    │          313 │
├─────────┴─────────┴──────────────┤
│ 12 rows                3 columns │
└──────────────────────────────────┘

In [38]:
pset5_prob3part2 = duckdb.query("WITH admitted AS  (\
                           SELECT Dept, Gender, sum(Freq) as Admitted_sum from ucb \
                           WHERE Admit = 'Admitted' \
                           GROUP BY Dept, Gender \
                           ) \
SELECT * FROM admitted")

pset5_prob3part2

┌─────────┬─────────┬──────────────┐
│  Dept   │ Gender  │ Admitted_sum │
│ varchar │ varchar │    int128    │
├─────────┼─────────┼──────────────┤
│ B       │ Female  │           17 │
│ C       │ Female  │          202 │
│ D       │ Female  │          131 │
│ E       │ Female  │           94 │
│ E       │ Male    │           53 │
│ A       │ Male    │          512 │
│ F       │ Female  │           24 │
│ A       │ Female  │           89 │
│ B       │ Male    │          353 │
│ C       │ Male    │          120 │
│ D       │ Male    │          138 │
│ F       │ Male    │           22 │
├─────────┴─────────┴──────────────┤
│ 12 rows                3 columns │
└──────────────────────────────────┘

In [42]:
pset5_prob3part3 = duckdb.query("SELECT Dept, Gender, Admitted_sum from pset5_prob3part2 \
             ")

pset5_prob3part3

┌─────────┬─────────┬──────────────┐
│  Dept   │ Gender  │ Admitted_sum │
│ varchar │ varchar │    int128    │
├─────────┼─────────┼──────────────┤
│ F       │ Female  │           24 │
│ B       │ Female  │           17 │
│ C       │ Female  │          202 │
│ D       │ Female  │          131 │
│ B       │ Male    │          353 │
│ C       │ Male    │          120 │
│ D       │ Male    │          138 │
│ F       │ Male    │           22 │
│ A       │ Male    │          512 │
│ A       │ Female  │           89 │
│ E       │ Female  │           94 │
│ E       │ Male    │           53 │
├─────────┴─────────┴──────────────┤
│ 12 rows                3 columns │
└──────────────────────────────────┘

In [57]:
pset5_prob3part4 = duckdb.query("SELECT pset5_prob3part3.Dept, pset5_prob3part3.Gender, pset5_prob3part3.Admitted_sum FROM pset5_prob3part3 \
                                JOIN pset5_prob3part2 ON \
                                pset5_prob3part2.Dept = pset5_prob3part3.Dept")

pset5_prob3part4

┌─────────┬─────────┬──────────────┐
│  Dept   │ Gender  │ Admitted_sum │
│ varchar │ varchar │    int128    │
├─────────┼─────────┼──────────────┤
│ B       │ Female  │           17 │
│ C       │ Female  │          202 │
│ D       │ Female  │          131 │
│ B       │ Male    │          353 │
│ C       │ Male    │          120 │
│ D       │ Male    │          138 │
│ E       │ Male    │           53 │
│ E       │ Female  │           94 │
│ A       │ Male    │          512 │
│ A       │ Female  │           89 │
│ ·       │  ·      │            · │
│ ·       │  ·      │            · │
│ ·       │  ·      │            · │
│ C       │ Male    │          120 │
│ D       │ Male    │          138 │
│ F       │ Male    │           22 │
│ F       │ Female  │           24 │
│ F       │ Male    │           22 │
│ F       │ Female  │           24 │
│ A       │ Male    │          512 │
│ A       │ Female  │           89 │
│ E       │ Male    │           53 │
│ E       │ Female  │           94 │
├